### 1. Avoids scroll-in-the-scroll in the entire Notebook

In [1]:
from IPython.display import Javascript
def resize_colab_cell():
    display(Javascript('google.colab.output.setIframeHeight(0, true, {maxHeight: 5000})'))
get_ipython().events.register('pre_run_cell', resize_colab_cell)

# 2. Functions for **`Embeddings`** and **`Cosine Similarity`** sores

### 2.1. Function to get Embeddings from Tokenized array

In [2]:
!pip install transformers

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 68.3 MB/s eta 0:00:00


In [3]:
import torch
import numpy as np
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')


def get_embeddings(tokenized_array):
    tokenized_array_input = " ".join(tokenized_array)
    tokenized_array_encoding = tokenizer(tokenized_array_input, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        tokenized_array_embedding = model(**tokenized_array_encoding).last_hidden_state.mean(dim=1).numpy()

    return tokenized_array_embedding

<IPython.core.display.Javascript object>

### 2.2. Function to get *`Cosine Similarity`*

In [4]:
def get_cosine_similarity(JD_req_skills_embeddings, JD_req_edu_embeddings, CV_skills_embeddings, CV_edu_embeddings):
    skills_score = cosine_similarity(JD_req_skills_embeddings, CV_skills_embeddings)
    edu_score = cosine_similarity(JD_req_edu_embeddings, CV_edu_embeddings)
    return (skills_score + edu_score) / 2.0

<IPython.core.display.Javascript object>

### 2.3. Function to get Cosine Similarity for Positions

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
tfidf_vectorizer = TfidfVectorizer()

<IPython.core.display.Javascript object>

In [6]:
def get_cosine_sim_for_category(position, category):
    tfidf_matrix = tfidf_vectorizer.fit_transform([position, category])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    return cosine_sim[0][0]

<IPython.core.display.Javascript object>

# 3. Preprocessing **`Job Description`**

### 3.1. Importing dataset from Hugging Face

In [7]:
!pip install datasets

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00


In [8]:
from datasets import load_dataset

dataset = load_dataset("jacob-hugging-face/job-descriptions")
print(dataset)

<IPython.core.display.Javascript object>

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['company_name', 'job_description', 'position_title', 'description_length', 'model_response'],
        num_rows: 853
    })
})


### 3.2. Extracting *`Company Name`*

In [9]:
def JD_extract_company(job_num):
    return dataset["train"][job_num]["company_name"]

<IPython.core.display.Javascript object>

### 3.3. Extracting *`Position`*

In [10]:
def JD_extract_position(job_num) :
    return dataset["train"][job_num]["position_title"]

<IPython.core.display.Javascript object>

### 3.4. Extracting *`Required Skills`*

In [11]:
def JD_extract_required_skills(job_num):
    model_response = eval(dataset["train"][job_num]["model_response"])
    return model_response["Required Skills"] if "Required Skills" in model_response else "N/A"

<IPython.core.display.Javascript object>

### 3.5. Extracting *`Educational Requirements`*

In [12]:
def JD_extract_educational_requirements(job_num):
    model_response = eval(dataset["train"][job_num]["model_response"])
    return model_response["Educational Requirements"] if "Educational Requirements" in model_response else "N/A"

<IPython.core.display.Javascript object>

# 4. Preprocessing **`Resume Data`**
> needs attention (to mount google drive & `.zip` file upload)

### 4.1. Uploading Resume PDFs as `Zip` and Extracting
> needs attention (to upload `.zip` file)

In [13]:
from google.colab import drive
drive.mount('/content/drive')

<IPython.core.display.Javascript object>

Mounted at /content/drive


In [14]:
import zipfile, os


# location/path of zip file in colab runtime storage
resume_folder = "/content/drive/MyDrive/Resume-to-Job-Matcher/dataset/resume_PDF_dataset.zip"

if zipfile.is_zipfile(resume_folder):
    with zipfile.ZipFile(resume_folder, 'r') as zip_ref:
        zip_ref.extractall()
        path = os.getcwd() + "/" + os.path.splitext(os.path.basename(resume_folder))[0]
else:
    print("# ERROR: Please upload a valid ZIP file with resume PDFs.")

<IPython.core.display.Javascript object>

### 4.2. Extracting text from resume PDF

In [15]:
!pip install PyPDF2

<IPython.core.display.Javascript object>

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


In [16]:
import PyPDF2


def extract_resume_text(filePath):
    reader = PyPDF2.PdfReader(open(filePath, "rb"))

    text = ""
    for i in range(len(reader.pages)): text += reader.pages[i].extract_text()
    return text

<IPython.core.display.Javascript object>

### 4.3. Extracting *`Category (Job Role)`*

In [17]:
def extract_category(text):
     return text.strip().split("\n")[0].strip() if text.strip() else None

<IPython.core.display.Javascript object>

### 4.4. Extracting *`Education`*

In [18]:
import re


def extract_education(text):
    education_keywords = [
        "High School", "Certificate", "Associate", "Diploma", "High School Diploma", "GED", "Undergraduate",
        "B.A.", "B.S.", "B.Sc.", "B.Eng.", "B.Tech.", "B.Pharm.", "B. Pharmacy", "B Pharmacy", "Bachelor", "BCA", "BBA", "Graduate", "B.",
        "M.A.", "M.S.", "M.Sc.", "M.Eng.", "M.Tech.", "M.Pharm.", "M. Pharmacy", "MBA", "MCA", "Master", "Postgraduate", "M.",
        "Ph.D.", "Doctorate", "Doctor", "Doctor of Medicine", "Doctor of Science", "D."
    ]

    education = []

    for word in education_keywords:
        pattern = r"(?i)\b{}\b".format(re.escape(word))
        match = re.search(pattern, text)
        if match: education.append(match.group())

    return education

<IPython.core.display.Javascript object>

### 4.5. Extracting *`Skills`*

In [19]:
!pip install spacy
!python -m spacy info
!python -m pip freeze | grep spacy
!python -m spacy download en_core_web_sm

<IPython.core.display.Javascript object>

2023-09-19 10:44:55.459467: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

============================== Info about spaCy ==============================

spaCy version    3.6.1                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-5.15.109+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        en_core_web_sm (3.6.0)        

en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl#sha256=83276fc78a70045627144786b52e1f2728ad5e29e5e43916ec37ea9c26a11212
spacy==3.6.1
spacy-legacy==3.0.12
spacy-loggers==1.0.4
2023-09-19 10:45:15.920489: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the pack

In [20]:
import spacy
nlp = spacy.load("en_core_web_sm")


def extract_skills(text):
    skills = []
    doc = nlp(text)

    for word in doc:
        if "NN" in word.tag_: skills.append(word.text)

    return list(set(skills))

<IPython.core.display.Javascript object>

# 5. Getting `Embeddings` for JD & CV
> processing time may differ based on the dataset size

### 5.1. For progress bar

In [21]:
!pip install tqdm
from tqdm import tqdm

<IPython.core.display.Javascript object>

### 5.2. For *`Job Description`*

In [22]:
JD_req_skills_embeddings_list = []
JD_req_edu_embeddings_list = []
JD_position_list = []

jobs = 10
for JD_num in tqdm(range(jobs), desc="Processing Jobs"):
    JD_req_skills_embeddings_list.append(get_embeddings(JD_extract_required_skills(JD_num)))
    JD_req_edu_embeddings_list.append(get_embeddings(JD_extract_educational_requirements(JD_num)))
    JD_position_list.append(JD_extract_position(JD_num))

<IPython.core.display.Javascript object>

Processing Jobs: 100%|██████████| 10/10 [00:05<00:00,  1.92it/s]


### 5.3. For *`Resume/CVs`*

In [23]:
import glob, os


CV_skills_embeddings_list = []
CV_education_embeddings_list = []
CV_category_list = []

pdf_files = glob.glob(os.path.join(path, "*.pdf"))

# for cv in pdf_files:
for cv in tqdm(pdf_files, desc="Processing resume/CVs"):
    thisPDF_text = extract_resume_text(cv)

    CV_skills_embeddings_list.append(get_embeddings(extract_skills(thisPDF_text)))
    CV_education_embeddings_list.append(get_embeddings(extract_education(thisPDF_text)))
    CV_category_list.append(extract_category(thisPDF_text))

<IPython.core.display.Javascript object>

Processing resume/CVs: 100%|██████████| 2484/2484 [49:31<00:00,  1.20s/it]


# 6. Driver Function
> processing time may differ based on the dataset size

In [26]:
import PyPDF2, glob, os


def main():
    # outer loop (jobs)
    for JD_num in tqdm(range(jobs), desc="Progress"):

        try:
            CV_and_score = []


            # inner loop (resume/CVs)
            for CV_num, cv in enumerate(pdf_files):
                try:
                    score = (get_cosine_similarity(JD_req_skills_embeddings_list[JD_num], JD_req_edu_embeddings_list[JD_num], CV_skills_embeddings_list[CV_num], CV_education_embeddings_list[CV_num]) + get_cosine_sim_for_category(JD_position_list[JD_num], CV_category_list[CV_num])) / 2.0
                    CV_and_score.append((score[0][0], os.path.basename(cv)))
                except:
                    print(f"\n# WARNING: Problematic Resume: {os.path.basename(cv)}\n")
            # inner loop end


            CV_and_score.sort(key=lambda x: x[0], reverse=True)
            top_scores = [score for score, _ in CV_and_score[:5]]
            top_CVs = [filename for _, filename in CV_and_score[:5]]

            print(f"\nCompany: {JD_extract_company(JD_num)} ({JD_position_list[JD_num]}): \nTop 5 CVs: {top_CVs} \nCorresponding Scores: {top_scores}\n")
            # i have kept the corresponding scores as it was, you can customise it accordingly (for example, score*100 to convert into %)

        except:
            print(f"\n# WARNING: Problematic Job Description: {JD_extract_company(JD_num)} ({JD_position_list[JD_num]})\n")

        print("\n>>>>>>>\n")
        # outer loop end

<IPython.core.display.Javascript object>

In [27]:
if __name__ == "__main__": main()

<IPython.core.display.Javascript object>

Progress:   0%|          | 0/10 [00:00<?, ?it/s]


# WARNING: Problematic Resume: 12632728.pdf



Progress:  10%|█         | 1/10 [00:08<01:15,  8.43s/it]


Company: Google (Sales Specialist): 
Top 5 CVs: ['15581242.pdf', '79376680.pdf', '29134721.pdf', '30083943.pdf', '29399491.pdf'] 
Corresponding Scores: [0.7054393, 0.5484214, 0.5450157, 0.54348207, 0.54214764]


>>>>>>>


# WARNING: Problematic Resume: 12632728.pdf



...


Company: Zander Insurance Agency (Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '68240723.pdf', '13014900.pdf', '76010167.pdf'] 
Corresponding Scores: [0.7980614, 0.7144139, 0.61715233, 0.6127536, 0.5925977]


>>>>>>>


# WARNING: Problematic Resume: 12632728.pdf



Progress:  90%|█████████ | 9/10 [01:22<00:09,  9.28s/it]


Company: Tuff (Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '68240723.pdf', '13014900.pdf', '76010167.pdf'] 
Corresponding Scores: [0.7993721, 0.71551275, 0.618664, 0.6139426, 0.5946326]


>>>>>>>


# WARNING: Problematic Resume: 12632728.pdf



Progress: 100%|██████████| 10/10 [01:31<00:00,  9.18s/it]


Company: General Dynamics Information Technology (SR. Web Designer): 
Top 5 CVs: ['29524570.pdf', '21283733.pdf', '34051710.pdf', '39776400.pdf', '76010167.pdf'] 
Corresponding Scores: [0.6068634, 0.5252452, 0.522153, 0.46799126, 0.45960188]


>>>>>>>

